In [1]:
import sys
from pathlib import Path
import json

import numpy as np
import pennylane as qml
from matplotlib import pyplot as plt
from qiskit import QuantumCircuit
from qiskit.utils import QuantumInstance
from qiskit_machine_learning.kernels import QuantumKernel
from qiskit.providers.aer import StatevectorSimulator
from qiskit.circuit.library import ZZFeatureMap
from sklearn.datasets import fetch_openml
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from torch.utils.tensorboard import SummaryWriter
from pandas import DataFrame

sys.path.extend(['/home/quic/QUIC-Projects/'])
from classifiers.quantum.qml.qasvm import PseudoTensorSoftQASVM
from classifiers.datasets.sklearn import SklearnDataset
from classifiers.quantum.ansatz import MultilayerCircuit9FeatureMap
from classifiers.convex.svm import CvxSoftQASVM

In [2]:
dataset = fetch_openml('mnist_784')
print('loaded mnist data set')

loaded mnist data set


In [7]:
test_size = 14780-2**13 #2**13
train_size = 2**13
n_feature = 10

In [8]:
def load_train_and_test_data(dataset, train_size:float=2**13, test_size:float=2**7):
    data = dataset.data.to_numpy()
    label = dataset.target.to_numpy().astype(float)
    # label = np.where(label%2, 1, 0)
    mask = (label==0) + (label==1)
    data = data[mask]
    label = label[mask]
    reproducible_seed = np.random.randint(99999)
    X_train, X_test, y_train, y_test = train_test_split(data, label, train_size=train_size, test_size=test_size, random_state=reproducible_seed)
    return X_train, y_train, X_test, y_test, reproducible_seed

def reduce_and_normalize_data(n_components, X_train, X_test):
    scaler = StandardScaler()
    pca = PCA(n_components=n_components)
    X_train = scaler.fit_transform(X_train)
    X_train = pca.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    X_test = pca.transform(X_test)
    for i, d in enumerate(X_train):
        X_train[i] = d/np.linalg.norm(d)
    for i, d in enumerate(X_test):
        X_test[i] = d/np.linalg.norm(d)
    return X_train, X_test

def construct_training_and_test_quantum_kernel_matrix_cpu(feature_map:QuantumCircuit, X_train:np.ndarray, X_test:np.ndarray):
    quantum_instance = QuantumInstance(backend = StatevectorSimulator(device='CPU')) # TODO: gpu vs cpu
    quantum_kernel = QuantumKernel(feature_map=feature_map, quantum_instance=quantum_instance, enforce_psd=False)
    training_kernel = quantum_kernel.evaluate(X_train, X_train)
    test_kernel = quantum_kernel.evaluate(X_train, X_test)
    return training_kernel, test_kernel

def construct_training_and_test_quantum_kernel_matrix_gpu(feature_map:QuantumCircuit, X_train:np.ndarray, X_test:np.ndarray):
    quantum_instance = QuantumInstance(backend = StatevectorSimulator(device='GPU')) # TODO: gpu vs cpu
    quantum_kernel = QuantumKernel(feature_map=feature_map, quantum_instance=quantum_instance, enforce_psd=False)
    training_kernel = quantum_kernel.evaluate(X_train, X_train)
    test_kernel = quantum_kernel.evaluate(X_train, X_test)
    return training_kernel, test_kernel

In [10]:
X_train, y_train, X_test, y_test, reproducible_seed = load_train_and_test_data(dataset, train_size=train_size, test_size=test_size)
X_train, X_test = reduce_and_normalize_data(n_feature, X_train, X_test)
feature_map = ZZFeatureMap(feature_dimension=n_feature, reps=2, entanglement='linear')

In [11]:
train_kernel, test_kernel = construct_training_and_test_quantum_kernel_matrix_cpu(feature_map=feature_map, X_train=X_train, X_test=X_test)

In [12]:
train_kernel, test_kernel = construct_training_and_test_quantum_kernel_matrix_gpu(feature_map=feature_map, X_train=X_train, X_test=X_test)

In [26]:
import pennylane as qml
from itertools import permutations, combinations

device = qml.device('lightning.gpu', wires = n_feature)

def qmlZZfeatureMap(params, wires:list, reps:int, entanglement:str='linear'):
    phi1 = lambda x: x
    phi2 = lambda x, y: (np.pi-x)*(np.pi-y)
    wires = list(wires)
    for wire in wires:
        qml.Hadamard(wires=wire)
        qml.U1(2.0*phi1(params[wire]), wires=wire)
    if entanglement=='linear':
        ent_wires = zip(wires[:-1], wires[1:])
    elif entanglement=='all':
        ent_wires = combinations(wires, 2)
    elif entanglement=='circular':
        ent_wires = zip(wires[:], wires[1:]+[wires[0]])
    for wire1, wire2 in ent_wires:
        qml.CNOT(wires=(wire1, wire2))
        qml.U1(2.0*phi2(params[wire1], params[wire2]), wires=wire2)
        qml.CNOT(wires=(wire1, wire2))


In [31]:
@qml.qnode(device=device)
def circuit(x1, x2):
    qmlZZfeatureMap(x1, wires=device.wires, reps=2)
    qml.adjoint(qmlZZfeatureMap)(x2, wires=device.wires, reps=2)
    return qml.probs(wires=device.wires)
kernel = lambda x1, x2: circuit(x1, x2)[0]

def construct_training_and_test_quantum_kernel_matrix_qml(X_train:np.ndarray, X_test:np.ndarray):
    training_kernel = qml.kernels.square_kernel_matrix(X_train, kernel)
    test_kernel = qml.kernels.kernel_matrix(X_train, X_test, kernel)
    return training_kernel, test_kernel

In [32]:
train_kernel, test_kernel = construct_training_and_test_quantum_kernel_matrix_qml(X_train=X_train, X_test=X_test)

KeyboardInterrupt: 

## Result

1. cpu: 19m 55.1s
2. gpu: 20m 27.7s
3. pennylane+gpu: >40m 8.4s